In [91]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [92]:
X,Y = make_classification(n_samples = 1000,n_features = 16,n_classes = 5,n_informative = 5)

In [93]:
Y= Y.reshape(-1,1)

In [94]:
X.shape,Y.shape,X[:10],Y[:10]

((1000, 16),
 (1000, 1),
 array([[ 2.41404011e+00, -3.01287593e-01,  2.28314573e+00,
          4.46496263e-01, -3.25942430e+00,  1.07472666e+00,
          6.91011224e-01,  1.06938957e+00,  3.16037213e-01,
         -3.53390752e-02,  9.30444431e-01, -5.36243463e-01,
          7.70201506e-01,  5.45843430e-02, -1.52371209e+00,
         -2.94961235e-01],
        [-1.55951757e+00,  1.14424361e+00,  2.88358232e-01,
          2.44035013e-01,  2.93053260e+00, -1.24705713e+00,
         -1.50666031e+00,  1.92551518e-01, -2.18125791e+00,
          2.28717199e+00,  1.30473003e+00, -1.67987587e-02,
          9.10420792e-01,  2.09629988e+00,  1.14286556e-01,
          4.38250404e-01],
        [ 2.54070229e+00, -5.82473265e-01,  2.81405150e-01,
         -1.87853761e+00, -1.45703037e+00, -8.88682816e-01,
         -2.09448133e+00, -3.90863085e-01,  3.85238016e+00,
         -3.08969570e+00,  1.81046465e+00, -6.54977770e-01,
          1.43543123e+00,  9.85475770e-01,  2.00158182e+00,
          1.59639399e

# 机器学习方法

In [95]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [96]:
reg = SVC()
reg.fit(X,Y.ravel())

SVC()

In [97]:
accuracy_score(reg.predict(X),Y)

0.84

# 用来做随机batch的模块

In [132]:
class DiabetesDataset(Dataset):
    def __init__(self, x_, y_):
        self.len = x_.shape[0]
        self.x_data = torch.from_numpy(x_)
        # 这部很关键， 特征类型要求float类型
        self.x_data = self.x_data.float()
        # 多分类的标签必须要用LongTensor类型
        self.y_data = torch.LongTensor(y_)


    def __getitem__(self, item):
        return self.x_data[item], self.y_data[item]

    def __len__(self):
        return self.len

In [171]:
dataset = DiabetesDataset(X, Y)
train_loader = DataLoader(dataset=dataset, batch_size=500, shuffle=True)

# 网络结构

In [172]:
#网络结构
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16, 32) 
        self.fc2 = nn.Linear(32, 10)  
        self.fc3 = nn.Linear(10, 5)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        return out

# 建立模型训练

In [173]:
model = NeuralNet()
#损失函数
criterion = nn.CrossEntropyLoss(reduction='mean')
#优化算法
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [179]:
total_step = len(train_loader)
num_epochs = 1000
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        x = x.reshape(-1, 16)
        # Forward pass
        outputs = model(x)
        #扁平化成一维的
        y = y.squeeze()
        loss = criterion(outputs, y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #输出中间信息
        if i % 50 == 0 and epoch%20==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/1000], Step [1/2], Loss: 0.0004
Epoch [21/1000], Step [1/2], Loss: 0.0003
Epoch [41/1000], Step [1/2], Loss: 0.0003
Epoch [61/1000], Step [1/2], Loss: 0.0003
Epoch [81/1000], Step [1/2], Loss: 0.0002
Epoch [101/1000], Step [1/2], Loss: 0.0002
Epoch [121/1000], Step [1/2], Loss: 0.0002
Epoch [141/1000], Step [1/2], Loss: 0.0002
Epoch [161/1000], Step [1/2], Loss: 0.0002
Epoch [181/1000], Step [1/2], Loss: 0.0002
Epoch [201/1000], Step [1/2], Loss: 0.0002
Epoch [221/1000], Step [1/2], Loss: 0.0002
Epoch [241/1000], Step [1/2], Loss: 0.0002
Epoch [261/1000], Step [1/2], Loss: 0.0002
Epoch [281/1000], Step [1/2], Loss: 0.0002
Epoch [301/1000], Step [1/2], Loss: 0.0002
Epoch [321/1000], Step [1/2], Loss: 0.0002
Epoch [341/1000], Step [1/2], Loss: 0.0002
Epoch [361/1000], Step [1/2], Loss: 0.0002
Epoch [381/1000], Step [1/2], Loss: 0.0001
Epoch [401/1000], Step [1/2], Loss: 0.0001
Epoch [421/1000], Step [1/2], Loss: 0.0001
Epoch [441/1000], Step [1/2], Loss: 0.0001
Epoch [461/1000],

# 预测

In [180]:
predicted = model(torch.LongTensor(X).float())

In [181]:
# 预测出来的结果要转换取每个数据最大值所在索引
predicted_class = predicted.argmax(dim=1)

In [182]:
accuracy_score(predicted_class.detach().numpy(),Y)

0.723